In [288]:
# Dependancies
import pandas as pd
import os
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from sqlalchemy import func
import warnings
warnings.filterwarnings('ignore')
from config import db_pass
import pymysql
from functools import reduce
from sqlalchemy import Column, Integer, String, Float
import matplotlib.pyplot as plt

In [241]:
# Establish the connection 
rds_connection_string = f"root:{db_pass}@127.0.0.1/stocks_db"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')
# conn = engine.connect()

                                PART 2
                                Data Minging
                                Cleaning the data, merging two dataframes for further analysis
                          

In [258]:
# Read in nyse data from the table
nyse_raw_df = pd.read_sql('SELECT * FROM companylist_nyse', con=engine)
nyse_raw_df

,id,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,SummaryQuote
0,1,DDD,3D Systems Corporation,10.7500,1250.00,None,Technology,Computer Software: Prepackaged Software,https://www.nasdaq.com/symbol/ddd
1,2,MMM,3M Company,190.2100,109640.00,None,Health Care,Medical/Dental Instruments,https://www.nasdaq.com/symbol/mmm
2,3,WBAI,500.com Limited,11.9700,508.97,2013,Consumer Services,Services-Misc. Amusement & Recreation,https://www.nasdaq.com/symbol/wbai
3,4,WUBA,58.com Inc.,71.6900,10630.00,2013,Technology,"Computer Software: Programming, Data Processing",https://www.nasdaq.com/symbol/wuba
4,5,EGHT,8x8 Inc,23.7000,2270.00,None,Technology,EDP Services,https://www.nasdaq.com/symbol/eght
5,6,AHC,A.H. Belo Corporation,3.7400,80.59,None,Consumer Services,Newspapers/Magazines,https://www.nasdaq.com/symbol/ahc
6,7,AOS,A.O Smith Corporation,55.8100,9370.00,None,Consumer Durables,Consumer Electronics/Appliances,https://www.nasdaq.com/symbol/aos
7,8,ATEN,"A10 Networks, Inc.",6.5800,494.71,2014,Technology,Computer Communications Equipment,https://www.nasdaq.com/symbol/aten
8,9,AAC,"AAC Holdings, Inc.",1.7200,42.45,2014,Health Care,Medical Specialities,https://www.nasdaq.com/symbol/aac
9,10,AIR,AAR Corp.,33.8100,1190.00,None,Capital Goods,Aerospace,https://www.nasdaq.com/symbol/air


In [243]:
# Drop unnecessary columns in nyse df
nyse_df = nyse_raw_df.drop(columns=['id', 'Name', 'LastSale', 'IPOyear', 'SummaryQuote'])
nyse_df.shape

(3112, 4)

In [244]:
# Drop Nan Values in nyse df
nyse_df = nyse_df.dropna()
nyse_df.shape

(1964, 4)

In [259]:
# Read in nasdaq data from the table
nasdaq_raw_df = pd.read_sql('SELECT * FROM companylist_nasdq', con=engine)
nasdaq_raw_df

,id,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,SummaryQuote
0,1,YI,"111, Inc.",8.6500,705.36,2018,Health Care,Medical/Nursing Services,https://www.nasdaq.com/symbol/yi
1,2,PIH,"1347 Property Insurance Holdings, Inc.",5.0650,30.45,2014,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pih
2,3,FLWS,"1-800 FLOWERS.COM, Inc.",19.4100,1250.00,1999,Consumer Services,Other Specialty Stores,https://www.nasdaq.com/symbol/flws
3,4,BCOW,"1895 Bancorp of Wisconsin, Inc.",9.6500,47.06,2019,Finance,Banks,https://www.nasdaq.com/symbol/bcow
4,5,VNET,"21Vianet Group, Inc.",8.0500,904.76,2011,Technology,"Computer Software: Programming, Data Processing",https://www.nasdaq.com/symbol/vnet
5,6,TWOU,"2U, Inc.",61.6100,3600.00,2014,Technology,Computer Software: Prepackaged Software,https://www.nasdaq.com/symbol/twou
6,7,QFIN,"360 Finance, Inc.",20.0200,2880.00,2018,Finance,Finance: Consumer Services,https://www.nasdaq.com/symbol/qfin
7,8,JOBS,"51job, Inc.",93.8700,5810.00,2004,Technology,Diversified Commercial Services,https://www.nasdaq.com/symbol/jobs
8,9,JFKKU,8i Enterprises Acquisition Corp,10.0700,50.35,2019,Finance,Business Services,https://www.nasdaq.com/symbol/jfkku
9,10,ABIL,Ability Inc.,1.4200,11.42,2014,Capital Goods,Telecommunications Equipment,https://www.nasdaq.com/symbol/abil


In [246]:
# Drop unnecessary columns
nasdaq_df = nasdaq_raw_df.drop(columns=['id', 'Name', 'LastSale', 'IPOyear', 'SummaryQuote'])

In [247]:
frames = [nyse_df, nasdaq_df]
result_df = pd.concat(frames)
result_df.head(2)

,Symbol,MarketCap,Sector,industry
0,DDD,1250.0,Technology,Computer Software: Prepackaged Software
1,MMM,109640.0,Health Care,Medical/Dental Instruments


In [249]:
# Check duplicates for tickers 
# result_df['Symbol'].value_counts()

In [250]:
# Check if the values are the same for each duplicate 
result_df[result['Symbol']=='PGTI']

,Symbol,MarketCap,Sector,industry
2194,PGTI,860.79,Capital Goods,Building Products
979,PGTI,860.79,Capital Goods,Building Products


In [251]:
# Dropping the symbol's duplicates
result_stock_df = result_df.drop_duplicates()
result_stock_df.head(2)

,Symbol,MarketCap,Sector,industry
0,DDD,1250.0,Technology,Computer Software: Prepackaged Software
1,MMM,109640.0,Health Care,Medical/Dental Instruments


                                                      PART
                                                      
                                       Uploading a new table into DataBase

In [252]:
# Transporting the dataframe to the table
result_stock_df.to_sql(name='stock_final_analysis', con=engine, if_exists='append', index=False)

In [253]:
# Check to see if the data was transformed 
pd.read_sql_query('select * from stock_final_analysis', con=engine).head()

,id,Symbol,MarketCap,Sector,industry
0,1,DDD,1250.00,Technology,Computer Software: Prepackaged Software
1,2,MMM,109640.00,Health Care,Medical/Dental Instruments
2,3,WBAI,508.97,Consumer Services,Services-Misc. Amusement & Recreation
3,4,WUBA,10630.00,Technology,"Computer Software: Programming, Data Processing"
4,5,EGHT,2270.00,Technology,EDP Services


In [254]:
# Analysing sectors in analysis table
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
# Check the DataBase for tables 
Base.classes.keys()

['all_stocks_5yr',
 'companylist_nasdq',
 'companylist_nyse',
 'stock_final_analysis']

In [256]:
# Save references to analysis table
Stock_final_analysis = Base.classes.stock_final_analysis

# Create a session
session = Session(engine)

# Create the inspector and connect it to the engine
inspector = inspect(engine)

# inspecting the columns
columns = inspector.get_columns('stock_final_analysis')   
for column in columns:
    print(column["name"], column["type"])

id INTEGER(11)
Symbol VARCHAR(30)
MarketCap DECIMAL(18, 2)
Sector VARCHAR(50)
industry VARCHAR(100)


                                                    PART 
                                        Sector & Industry Analysis

In [260]:
# Filter by Sector to reveal which sector has the most shares 
sector = session.query(Stock_final_analysis.Sector, func.count(Stock_final_analysis.Symbol)).\
    group_by(Stock_final_analysis.Sector).\
    order_by(func.count(Stock_final_analysis.Symbol).desc()).all()
print(sector)

[('Consumer Services', 1146), ('Health Care', 1140), ('Finance', 960), ('Technology', 902), ('Capital Goods', 522), ('Energy', 456), ('Basic Industries', 430), ('Public Utilities', 328), ('Consumer Non-Durables', 300), ('Miscellaneous', 206), ('Consumer Durables', 168), ('Transportation', 160)]


In [294]:
# Create a list of dictionaries in order to transform thr data into DataFrame 
sector_list = []
for key, value in sector:
    sector_dict = {}
    sector_dict[key] = value         
    sector_list.append(sector_dict)    
print(sector_list)

# for key, value in sector:
#     sector_dict = {}
#     sector_dict[key] = value         
#     print(sector_dict)

[{'Consumer Services': 1146}, {'Health Care': 1140}, {'Finance': 960}, {'Technology': 902}, {'Capital Goods': 522}, {'Energy': 456}, {'Basic Industries': 430}, {'Public Utilities': 328}, {'Consumer Non-Durables': 300}, {'Miscellaneous': 206}, {'Consumer Durables': 168}, {'Transportation': 160}]


In [295]:
pd.DataFrame(sector_list)


,Basic Industries,Capital Goods,Consumer Durables,Consumer Non-Durables,Consumer Services,Energy,Finance,Health Care,Miscellaneous,Public Utilities,Technology,Transportation
0,NaN,NaN,NaN,NaN,1146.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1140.0,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,960.0,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,902.0,NaN
4,NaN,522.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,456.0,NaN,NaN,NaN,NaN,NaN,NaN
6,430.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,328.0,NaN,NaN
8,NaN,NaN,NaN,300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,206.0,NaN,NaN,NaN


In [277]:
# How much money is in the Each Sector
fin = session.query(Stock_final_analysis.Sector, func.sum(Stock_final_analysis.MarketCap)).\
    group_by(Stock_final_analysis.Sector).\
    order_by(func.count(Stock_final_analysis.Symbol).desc()).all()
print(fin)

[('Consumer Services', Decimal('10557686.76')), ('Health Care', Decimal('8749321.36')), ('Finance', Decimal('11111801.14')), ('Technology', Decimal('15182502.18')), ('Capital Goods', Decimal('5102864.44')), ('Energy', Decimal('5697280.80')), ('Basic Industries', Decimal('4235894.32')), ('Public Utilities', Decimal('5394447.42')), ('Consumer Non-Durables', Decimal('4375217.54')), ('Miscellaneous', Decimal('3534233.88')), ('Consumer Durables', Decimal('836981.54')), ('Transportation', Decimal('1951031.50'))]


In [269]:
# Filter by Industry to reveal what industry has the most shares 
industry = session.query(Stock_final_analysis.industry, func.count(Stock_final_analysis.industry)).\
    filter(Stock_final_analysis.Sector == 'Consumer Services').\
    group_by(Stock_final_analysis.industry).\
    order_by(func.count(Stock_final_analysis.industry).desc()).all()
print(industry)

[('Real Estate Investment Trusts', 364), ('Other Consumer Services', 102), ('Restaurants', 78), ('Other Specialty Stores', 60), ('Clothing/Shoe/Accessory Stores', 58), ('Hotels/Resorts', 56), ('Marine Transportation', 40), ('Broadcasting', 36), ('Services-Misc. Amusement & Recreation', 32), ('Department/Specialty Retail Stores', 32), ('RETAIL: Building Materials', 30), ('Catalog/Specialty Distribution', 28), ('Transportation Services', 22), ('Television Services', 18), ('Professional Services', 18), ('Telecommunications Equipment', 16), ('Movies/Entertainment', 14), ('Food Chains', 14), ('Military/Government/Technical', 14), ('Newspapers/Magazines', 14), ('Diversified Commercial Services', 12), ('Home Furnishings', 10), ('Advertising', 8), ('Consumer Electronics/Video Chains', 8), ('Real Estate', 8), ('Rental/Leasing Companies', 6), ('Office Equipment/Supplies/Services', 6), ('Building operators', 6), ('Books', 6), ('Farming/Seeds/Milling', 4), ('Homebuilding', 4), ('Recreational Produ